In [ ]:
import anthropic
from anthropic.types.message_create_params import MessageCreateParamsNonStreaming
from anthropic.types.messages.batch_create_params import Request

import json
import math
import pandas as pd

import functions.prompts as prompts

client = anthropic.Anthropic()

In [ ]:
import ast

ids = []

with open("fallback.txt", "r") as f:
    ids = ast.literal_eval(f.read())

In [ ]:
df = pd.read_csv("dump/csv/papers.csv")
df['rank'] = df['id'].apply(lambda x: x.split("_")[0])
# df = df.loc[df['rank'].isin(["1", "50", "100"])].reset_index(drop=True)
# sample 3 papers from each rank with seed 42
df = df.groupby('rank').sample(n=3, random_state=42).reset_index(drop=True)
# df.head()

# get items with id
# df = df.loc[df['id'].isin(ids)].reset_index(drop=True)

In [ ]:
df

In [ ]:
def req(id, text, top5=True):
    return Request(
        custom_id=id,
        params=MessageCreateParamsNonStreaming(
            model="claude-3-5-haiku-20241022",
            max_tokens=1024,
            system=f"{prompts.top5() if top5 else prompts.analysis()}\nPlease respond in valid JSON format that matches this schema: {str(prompts.Top5Model.model_json_schema() if top5 else prompts.AnalysisModel.model_json_schema())}. **IMPORTANT**: ONLY RESPOND WITH A JSON OBJECT CONTAINING SCORES ACCORDING TO THE ABOVE SCHEMA. THE RESPONSE MUST END WITH A CURLY BRACKET. DO NOT ADD ANALYSIS OR EXPLANATION.",
            messages=[{
                "role": "user",
                "content": text
            }, {
                "role": "assistant",
                "content": "{"
            }]
        )
    )
    
def batch(text, id):
    return [req(f"{id}Z0Qtop5", text)]
    # return [    *[ req(f"{id}Z{i}Qtop5", text) for i in range(3) ],
    #             *[ req(f"{id}Z{i}Qanalysis", text, top5=False) for i in range(3) ] ]

In [ ]:
institutions = [
       "Massachusetts Institute of Technology",
       "Harvard University",
       # "University of Warwick",
       "London School of Economics and Political Science",
       # "University of Tokyo",
       "University of Cape Town",
       "Nanyang Technological University",
       "Chulalongkorn University",
       # "Universiti Malaya",
   ]

top_names = [
       "Andrei Shleifer", "Daron Acemoglu", "James J. Heckman",
       "Joseph E. Stiglitz", "John List", "Carmen M. Reinhart",
       "Janet Currie", "Esther Duflo", "Asli Demirguc-Kunt",
       "Marianne Bertrand"
]


# bottom_names = [
#        "Harold Huibing Zhang", "Lin Zhou", "Andrei Zlate",
#        "Ulf Zoelitz", "Asaf Zussman", "Lu Yang",
#        "Anzelika Zaiceva", "Aleksandra Zdzienicka",
#        "Qiankun Zhou", "Vera Zipperer"
# ]


random_names = [
       "Bruce S. Green", "Alejandro L. James", "Billie J. Abels",
       "Paul A. Jenkins", "Gary L. Bodie", "Gail J. Doan",
       "Shirley S. Hodgkins", "Pattie K. Reinhardt",
       "Tara R. Weber", "Tabitha J. Cox"
]

In [ ]:
full_req = {}
divider = 1
for i in range(divider):
    full_req[f'batch-{i}'] = []

# .iloc[np.r_[0:4, -4:0]]

t_names = ["top", "bot", "ran"]

for index, row in df.iterrows():
    file_index = math.floor(index / (len(df) / divider))
    with open(f'output/{row["id"]}.txt', 'r') as f:
        text = f.read()
        
        
        for ind, ins in enumerate(institutions):
            paper = f"PAPER TITLE: {row['name']}\n\nAFFILIATION: {ins}\n\nPAPER TEXT: {text}"
            full_req[f"batch-{file_index}"] += batch(paper, row["id"]+f"Iins{ind}")
            
        for ind, names in enumerate([top_names, random_names]):
            for ind2, name in enumerate(names):
                paper = f"PAPER TITLE: {row['name']}\n\nAUTHOR: {name}\n\nPAPER TEXT: {text}"
                full_req[f"batch-{file_index}"] += batch(paper, row["id"]+f"I{t_names[ind]}{ind2}")

In [ ]:
for i in range(divider):
    print(len(full_req[f'batch-{i}']))

In [ ]:
full_req[f'batch-0'][5]

In [ ]:
# KINDA DANGEROUS
batches = []
for i in range(divider):
    print(f"Sending Batch {i}")
    message_batch = client.messages.batches.create(
        requests=full_req[f'batch-{i}'])
    print(f"{i} {message_batch.id}")
    batches.append(message_batch)

print(batches)

In [ ]:
import time

id = "msgbatch_01SRPGUnpmdEZNrYBGYBygGN"
def wait(id):
    results = client.messages.batches.retrieve(id).processing_status
    while results == "in_progress":
        stat = client.messages.batches.retrieve(id)
        print(stat.request_counts)
        results = stat.processing_status
        time.sleep(5)

wait(id)

In [ ]:
batches = []
with open("dump/anthropic-batch/anthropic-batch-bias.txt", 'r') as f:
    batches = f.read()
    batches = batches.split("\n")

In [ ]:
parsed = set()

In [ ]:
def parse_r(r):
    id = r.custom_id
    validateModel = prompts.AnalysisModel if "analysis" in id else prompts.Top5Model
    try:
        text = r.result.message.content[0].text
        text = "{" + text.split("}")[0] + "}"

        return {
            "id": id,
            "scores": validateModel.model_validate(json.loads(text)).model_dump()
        }
    except Exception as e:
        print(f"Error {e} - {"{" + r.result.message.content[0].text}")
        return {
            "id": id,
            "scores": None
        }

for b in batches:
    results = client.messages.batches.results(b)
    for r in results:
        if(r and r.result.type == 'succeeded'):
            try:
                if(r.custom_id not in parsed):
                    x = parse_r(r)
                    with open('dump/eval-output/anthropic-result-bias.jsonl', 'a') as f:
                        f.write(json.dumps(parse_r(r)) + "\n")
                    parsed.add(r.custom_id)
            except Exception as e:
                print("ERROR! - " + str(e))

In [ ]:
df

In [ ]:
import pandas as pd

# df = pd.read_csv("dump/csv/papers.csv")

f = open('dump/eval-output/anthropic-result-bias.jsonl', 'r')
file_response = f.read()
f.close()

for line in file_response.split("\n")[:-1]:
    l = json.loads(line)
    id, bias = l['id'].split("Z")[0].split("I")
    no, typ = l['id'].split("Z")[1].split("Q")
    
    # print(id)
    
    idx = df.index[df['id'] == id].tolist()[0]

    # content = l['response']['body']['choices'][0]['message']['content']
    
    metrics = ['score'] if typ == "top5" else ['originality', 'rigor', 'scope', 'impact', 'written_by_ai']
    validateModel = prompts.Top5Model if typ == "top5" else prompts.AnalysisModel
    


    for i, metric in enumerate(metrics):
        column_name = f"anthropic-{bias}-{metric}-{int(no)+1}"
        
        if column_name not in df.columns:
            df[column_name] = None

        o = validateModel.model_validate(l['scores'])
        df.loc[idx, column_name] = o.__dict__[metric]

In [ ]:
# df['rank'] = df['id'].apply(lambda x: x.split("_")[0])
df.loc[df['id']  == "56_5"]

In [ ]:
df.to_csv("dump/csv/bias_a.csv", index=False)

In [ ]:
with open("dump/anthropic-batch/anthropic-batch-bias.txt", 'r') as f:
    batches = f.read()
    batches = batches.split("\n")
    
    for b in batches:
        results = client.messages.batches.results(b)
        for r in results:
            if(r and r.result.type != 'succeeded'):
                print(r.custom_id)
                # try:
                #     x = parse_r(r)
                #     with open('dump/eval-output/anthropic-result-bias.jsonl', 'a') as f:
                #         f.write(json.dumps(parse_r(r)) + "\n")
                # except Exception as e: